## Importing Packages

In [1]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Packages Used ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

#------------------------To solve the ODE----------------------------
using DifferentialEquations
#using LSODA

#------------------------For Calling Python---------------------------
using PyCall
np = pyimport("numpy")
using SciPy

#------------------------Writing Data to CSV file---------------------
using CSV
using Tables

## Density Functions

In [5]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Defining the System ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


#---------------------------function to calculate r_isco------------------------

function r_isco(m)    
        """
        Radius of the Innermost Stable Circular Orbit (ISCO) of a Schwarzschild black hole with mass m
        """
    return 6.0*float(m)
end


#---------------------------Initial Conditions-----------------------------------

function initial_conditions(mass, a0, e0, phi0 = 0.)

        """
        Calculate the initial conditions for a Keplerian orbit with parameters a, e
        """
    r0 = a0 * (1. - e0^2) / (1. + e0 * cos(phi0))
    dphi0 = sqrt(mass[3] * a0 * (1. - e0^2)) / r0^2
    dr0 = a0* (1. - e0^2) / (1 + e0*cos(phi0))^2 * e0 *sin(phi0)*dphi0
    return [r0, phi0, dr0, dphi0]
end

#-------------------------Halo Models--------------------------------------------

function Spike_Halo(r, rho_spike, r_spike, alpha, r_min=0.)   # Spike Model

        """
        Defining the Halo Model from Eda paper [https://arxiv.org/abs/1408.3534]

        The density is given by:
            rho (r) = rho_spike*(r_spike/r)^(alpha)

        Parameters:
            rho_spike : float
                The density parameter of the spike profile
            r_spike : float
                The scale radius of the spike profile
            alpha : float
                The power-law index of the spike profile, with condition 0 < alpha < 3
        """

    return  ifelse.(r .> r_min, rho_spike.*(r_spike./r).^alpha, 0.)
end


function Const_Halo(r, rho_0, r_min=0.)    # Constant Density Model
        """
        Constant Halo with density rho_0
        """
    return  ifelse.(r .> r_min, rho_0, 0.)
end


#---------------------------System Parameters-----------------------------------
m1 = 4.8e-9
m2 = 4.8e-14
D  = 1e5 
a0 = 100.0*r_isco(m1)                
e0 = 0.

#---------------------------Mass Array------------------------------------------
m_total = m1+m2
mu = (m1*m2)/(m1+m2)
mass=[m1,m2,m_total,mu]

#--------------------Halo Parameters from Eda paper-----------------------------
rho_spike = 1.0848e-11               # [226.*solar_mass_to_pc]
alpha_spike = 2.544
r_spike = 0.54


u0 = initial_conditions(mass, a0, e0)



#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Evolving the System ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

#-------------------------Evolution Parameters--------------------------------------------

F0 = sqrt(mass[3]/a0^3)/2/pi
nOrbits = 1000                     # Number of Orbits
tp_per_orb = 100                   # Time-points per orbit
dp = nOrbits*tp_per_orb            # total number of data points the code runs for
t_end = nOrbits/F0

0.4432470050744553

In [6]:
print(F0)

2256.07841350676

In [7]:
print(mass)

[4.8e-9, 4.8e-14, 4.800048e-9, 4.799952000479995e-14]

## Solving the Differential Equations

In [3]:
#-------------------------Evolution Function-----------------------------------------------

function Evolve(mass, u0, t_end, t_start=0., gwEmission=true, dynamicalFriction=true, PostNewtonian=true, coulombLog=3., nSteps=1000)
        
        """
        Evolve the system of differential equations from t_start to t_end with initial conditions u0
        """
    t_eval = LinRange(t_start, t_end, nSteps)
    eta    = mass[4]/mass[3]
    m      = mass
    tspan  = (t_start,t_end)

    
    #---------------------------Derivative Function---------------------------------------
    
    function orbit!(du, u, m, t)
        
        # r=u[1],  phi=u[2],  dr=u[3],  dphi=u[4]
        # m = [m1,m2,m_total,mu]
        
        v = sqrt(u[3]^2 + (u[1]^2)*(u[4])^2) 
        
        P_df     = dynamicalFriction ? 4*pi*(m[2]^2 / m[4]) * (Spike_Halo(u[1],rho_spike, r_spike, alpha_spike)) * (coulombLog/v^3) : 0.
        
        P_pn_r   = PostNewtonian ? (m[3]/u[1]^2)*( (4 + 2*eta)*(m[3]/u[1]) - (3*eta +1)*(u[1]^2)*(u[4]^2) + (3 - 7*eta/2)*u[3]^2) : 0.
        
        P_pn_phi = PostNewtonian ? (m[3]/u[1]^2)*(4 - 2*eta)*u[3]*u[4] : 0.
        
        P_gw_r   = gwEmission ? ( (8/5)*m[4]*(m[3]/u[1]^3)*(2*v^2 + (8/3)*(m[3]/u[1]) )) : 0.
        
        P_gw_phi = gwEmission ? ( (8/5)*m[4]*(m[3]/u[1]^3)*(v^2 + 3*(m[3]/u[1]) )) : 0.
        
        
        du[1] = u[3]  # dr
        du[2] = u[4]  # dphi
        du[3] = -(-P_gw_r + P_df)*(u[3]) + P_pn_r - (m[3]/u[1]^2) + u[1]*(u[4]^2)  # ddr
        du[4] = -(P_gw_phi + P_df)*(u[4]) + P_pn_phi - (2*u[3]*u[4]/u[1])          # ddphi
    end
    
    #---------------------Call-back functions---------------------------------------------
    
    function terminate_condition(u,t,integrator)       # condition at which the integration terminates
        u[1]< 7*r_isco(m1)
    end
    
    function terminate_affect!(integrator)
        terminate!(integrator)
    end
    
    terminate_cb =DiscreteCallback(terminate_condition,terminate_affect!)
 
    
    #-------------------------Calling the solver--------------------------------------------
    
    # call-backs do not work with lsoda() but they do work with other algorithms
    
    prob = ODEProblem(orbit!, u0, tspan, m, callback=terminate_cb)  # exclude the callback part when using lsoda()
    alg = VCABM() #lsoda() #AN5() #VCABM5() #Tsit5()  #DP5() #AutoVern7(Rodas5())  #Vern9(lazy=false) #Feagin12() #Vern7() 
    @time sol = solve(prob, alg, abstol=1e-14, reltol=1e-12, saveat=t_eval, dense=false, maxiters=Int(1e9))    # For predefined time-steps 
    #@time sol = solve(prob, alg, abstol=1e-14, reltol=1e-12, dense=true, maxiters=Int(1e9))                   # For adaptive time-steps
    return sol
end


Evolve (generic function with 7 methods)

In [4]:
solution = Evolve(mass,u0,t_end,0.,true,true,true,3., dp)

sol_trans = solution'
r = sol_trans[:,1]
phi = sol_trans[:,2]
dr = sol_trans[:,3]
dphi = sol_trans[:,4]
t = solution.t

solution = nothing
sol_trans = nothing


  2.934117 seconds (8.70 M allocations: 381.392 MiB, 3.78% gc time, 95.42% compilation time)


## Calculating GW strain

In [14]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~Calculating the GW strain~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


#--------------------------------Strain Function------------------------------------------------

function strain(mass, D, r, phi, t)
    """
    This function takes in the orbit solution arrays (r and phi) and returns the gravitational wave pattern 
    """
    
    #---------Observer parameters--------------
    mu = mass[4]
    theta_o = 0    # inclination_angle
    phi_o = 0      # pericenter_angle
    
    #---------Scaling--------------------------
    r0 = u0[1]
    r /= r0
    T = t[2]-t[1]
    
    #---------Rotating body parameters---------
    x = r.*cos.(phi)
    y = r.*sin.(phi)
    z = zeros(length(x))
    
    #--------Quadrupole Moment Tensor----------
    Q  = [ [mu*x.*x, mu*x.*y, mu*x.*z],
           [mu*y.*x, mu*y.*y, mu*y.*z],
           [mu*z.*x, mu*z.*y, mu*z.*z] ]
    
    #--------Derivative Functions---------------
    
    function Mdt(Q)     
        """
        Returns the first derivative of Quadrupole Moment Tensor with respect to time
        """
        
        dQdt = [ [np.gradient(Q[1][1], T), np.gradient(Q[1][2], T), np.gradient(Q[1][3], T)], 
                 [np.gradient(Q[2][1], T), np.gradient(Q[2][2], T), np.gradient(Q[2][3], T)],
                 [np.gradient(Q[3][1], T), np.gradient(Q[3][2], T), np.gradient(Q[3][3], T)] ]
        
        return dQdt
    end
    
    
    function Mdt2(Q) 
        """
        Returns the second derivative of Quadrupole Moment Tensor with respect to time
        """
        
        dQ2dt2 = Mdt(Mdt(Q))
        
        return dQ2dt2*(r0^2)
    end
    
    
    d2Qd2t = Mdt2(Q)
    
    
    h_plus =  (1.0/D) * (   d2Qd2t[1][1].*(cos.(phi_o).^2 - sin.(phi_o).^2 .*cos.(theta_o).^2) 
                          + d2Qd2t[2][2].*(sin.(phi_o).^2 - cos.(phi_o).^2 .*cos.(theta_o).^2) 
                          - d2Qd2t[3][3].*(sin.(theta_o).^2) 
                          - d2Qd2t[1][2].*(sin.(2*phi_o).*(1.0 .+ cos.(theta_o).^2))
                          + d2Qd2t[1][3].*(sin.(phi_o).*sin.(2*theta_o)) 
                          + d2Qd2t[2][3].*(cos.(phi_o).*sin.(2*theta_o))     ) 
    
    h_cross = (1.0/D) * (   (d2Qd2t[1][1]-d2Qd2t[2][2]).*sin.(2*phi_o).*cos.(theta_o)
                             + 2*d2Qd2t[1][2].*cos.(2*phi_o).*cos.(theta_o) 
                             - 2*d2Qd2t[1][3].*cos.(phi_o).*sin.(theta_o) 
                             + 2*d2Qd2t[2][3].*sin.(theta_o).*sin.(phi_o)    )
    
    return [h_plus, h_cross]
    
end


function strain2d(mass, D, r, phi, t)
    
    """
    This function takes in the orbit solution arrays (r and phi) and returns the gravitational wave pattern 
    """
    
    #-------------Observer parameters----------
    mu = mass[4]
    theta_o = 0    # inclination_angle
    phi_o = 0      # pericenter_angle
    
    #----------------Scaling-------------------
    r0 = u0[1]
    r /= r0
    T = t[2]-t[1]
    
    #---------Rotating body parameters---------
    x = r.*cos.(phi)
    y = r.*sin.(phi)
    
    #--------Quadrupole Moment Tensor----------
    Q  = [ [mu*x.*x, mu*x.*y],
           [mu*y.*x, mu*y.*y] ]
    
    
    #--------Derivative Functions---------------

    function Mdt(Q)     
        """
        Returns the first derivative of Quadrupole Moment Tensor with respect to time
        """
        
        dQdt = [ [np.gradient(Q[1][1], T), np.gradient(Q[1][2], T)], 
                 [np.gradient(Q[2][1], T), np.gradient(Q[2][2], T)], ]
        
        return dQdt
    end
    
    
    function Mdt2(Q) 
        """
        Returns the second derivative of Quadrupole Moment Tensor with respect to time
        """
        
        dQ2dt2 = Mdt(Mdt(Q))
        
        return dQ2dt2*(r0^2)
    end
    
    
    d2Qd2t = Mdt2(Q)
    
    
    h_plus =  (1.0/D) * (   d2Qd2t[1][1].*(cos.(phi_o).^2) 
                          + d2Qd2t[2][2].*(-cos.(phi_o).^2 .*cos.(theta_o).^2)  )
    
    return h_plus
    
end


#--------------------------------FFT Function------------------------------------------------

function strainFFT(t, strain, f_bin)
    N = length(t)
    T = t[2] - t[1]
    
    h_plus_fft = SciPy.fft.fft(strain[1,:])/(2*pi*N)
    h_cross_fft = SciPy.fft.fft(strain[2,:])/(2*pi*N)
    xf = SciPy.fft.fftfreq(N, T)  #[1:Int(N ÷ 2)] 
    
    h_plus_fft = h_plus_fft[(xf .> f_bin[1]) .&& (xf .< f_bin[2])]
    h_cross_fft = h_cross_fft[(xf .> f_bin[1]) .&& (xf .< f_bin[2])]
    xf = xf[(xf .> f_bin[1]) .&& (xf .< f_bin[2])]

    return [xf, h_plus_fft, h_cross_fft]
end


function strainFFT2d(t, strain, f_bin)
    N = length(t)
    T = t[2] - t[1]
    
    h_plus_fft = SciPy.fft.fft(strain[:])/(2*pi*N)
    xf = SciPy.fft.fftfreq(N, T) #[1:Int(N ÷ 2)]
    
    h_plus_fft = h_plus_fft[(xf .> f_bin[1]) .&& (xf .< f_bin[2])]
    xf = xf[(xf .> f_bin[1]) .&& (xf .< f_bin[2])]
    
    return [xf , h_plus_fft]
end


strainFFT2d (generic function with 1 method)

In [15]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~Calculating the Strain~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
hz_to_invpc = 1.029e8
gw_strain  = strain2d(mass, D, r, phi, t);
strain_FFT  = strainFFT2d(t, gw_strain, [1e-4*hz_to_invpc, 1e-1*hz_to_invpc]);


## Producing Datasets

In [16]:
# semi-major axis data

v   = sqrt.(dr.^2 + (r.^2 .* dphi.^2))
sma = m_total*(1 ./abs.(v.^2 - 2*m_total ./r)) 

sma_data = hcat(t[:],sma[:])     # Stacking the matrices together

#-----------------------------------------------------------------------------
CSV.write("semi_major_axis_spike_Orb1k_dp100_alpha_2point50.csv", Tables.table(sma_data), writeheader=false)

#--------------------------To release memory----------------------------------
v = nothing
sma = nothing
sma_data = nothing;

In [17]:
# GW Strain Data

gw_data = hcat(t[:],gw_strain[:])

#-----------------------------------------------------------------------------
CSV.write("GW_strain_spike_Orb1k_dp100_alpha_2point50.csv", Tables.table(gw_data), writeheader=false)

#--------------------------To release memory----------------------------------
gw_data = nothing

In [18]:
# Characteristic Strain Data

f_start   = 1
f_end     = length(strain_FFT[1])
frequency = abs.(strain_FFT[1][f_start:f_end])/hz_to_invpc
h_char    = 2*abs.(strain_FFT[1][f_start:f_end]) .* abs.(strain_FFT[2][f_start:f_end])

char_strain_data = hcat(frequency[:], h_char[:])

#-----------------------------------------------------------------------------
CSV.write("char_strain_spike_Orb1k_dp100_alpha_2point50.csv", Tables.table(char_strain_data), writeheader=false)

#--------------------------To release memory----------------------------------

frequency = nothing
h_char = nothing
char_strain_data = nothing